In [45]:
import pandas as pd
import os
import argparse
from itertools import chain
from collections import defaultdict
import torch

from fsfp import config
from fsfp.utils.data import trunc_sequence
from fsfp.retrieval import Retriever

Step 1. Preprocess data <br>
Learning objective: 
1. Structure of raw data
2. Structure of data after "preprocess.py"
3. Recreate the preprocess steps

Data preprocessing <br>
The datasets of ProteinGym should be put under `data/substitutions/`. Run `python preprocess.py -s` to preprocess the raw datasets and pack them to `data/merged.pkl`.

In [ ]:
## In preprocess.py

# data_path = config.data_path.replace('.pkl', f'{args.save_postfix}.pkl') <- replace file suffix(.pkl) with user suffix with user-defined suffix in filename 
# datasets = merge_files(config.raw_data_dir, shuffle=args.shuffle, max_len=args.max_length, save_path=data_path) 

In [ ]:
# def merge_files(data_dir, shuffle=True, max_len=1022, save_path=None):
#     file_names = os.listdir(data_dir)
#     proteins = defaultdict(list) # Declare that the values in this dict are lists
#     for file_name in file_names:
#         if 'indels' in file_name:
#             continue # If "indel" in file_name, skip the file
#         protein = process_dms(f'{data_dir}/{file_name}', shuffle, max_len) # Carry out "process_dms" function on the csv file 
#         name = '_'.join(file_name.split('_')[:2]) # Extract the first 2 parts of the file name that is separated by underscore
#         proteins[name].append(protein) # Add a key-value pair: key - name, value - output of "process_dms" function 
                                         # Final data structure of proteins: dict: key-name, value: list of dictionary 
                                         # Value dict: keys(['wild_type', 'df', 'n_sites', 'name', 'offset']) 
                                         # wile_type aa seqeuence
                                         # df containing ['wt_aas', 'mt_aas', 'positions', 'DMS_score', 'DMS_score_bin'], index: 'mutant'
                                         # n_sites: number of mutations (e.g. for ssvl, it would be 1)
                                         # name: csv dataset name
                                         # offset: for truncation  
    
#     if save_path is not None:
#         make_dir(save_path) 
#         torch.save(proteins, save_path) # Same the dictionary in "merged.pkl"
#     return proteins

In [2]:
folder_path = "/illumina/scratch/SG_PROTEIN_ENG/iPEG/Liuyi/Pro-FSFP/"
data_dir = os.path.join(folder_path, config.raw_data_dir)
file_names = os.listdir(data_dir)
print(file_names) # Currently there are 2 datasets in "data/substitutions"
csv = file_names[1]
df = pd.read_csv(data_dir + "/" + csv, index_col='mutant')
print(df.columns, len(df))

df = df.sample(frac=1)
df.head()

['SYUA_HUMAN_Newberry_2020.csv', 'YAP1_HUMAN_Araya_2012.csv']
Index(['mutated_sequence', 'DMS_score', 'DMS_score_bin',
       'Tranception_L_no_retrieval', 'Tranception_S_retrieval',
       'Tranception_M_retrieval', 'Tranception_L_retrieval', 'EVE_single',
       'EVE_ensemble', 'MSA_Transformer_single', 'MSA_Transformer_ensemble',
       'ESM1v_single', 'ESM1v_ensemble', 'Wavenet', 'DeepSequence_single',
       'DeepSequence_ensemble', 'Site_Independent', 'EVmutation', 'RITA_s',
       'RITA_m', 'RITA_l', 'RITA_xl', 'RITA_ensemble', 'Progen2_small',
       'Progen2_medium', 'Progen2_base', 'Progen2_large', 'Progen2_xlarge',
       'Progen2_ensemble', 'Unirep', 'Unirep_evotune', 'GEMME', 'VESPA',
       'VESPAl', 'ESM1b', 'ProtGPT2', 'TranceptEVE_L'],
      dtype='object') 10075


,mutated_sequence,DMS_score,DMS_score_bin,...,ESM1b,ProtGPT2,TranceptEVE_L
mutant,,,,,,,
E178A:M179I,MDPGQQPPPQPAPQGQ...,0.857,1,...,-19.548,-6.515,-0.046
E178A:L190R,MDPGQQPPPQPAPQGQ...,0.659,1,...,-25.905,-6.483,-0.058
D170K:Q200R,MDPGQQPPPQPAPQGQ...,4.720,1,...,-27.366,-6.508,-0.042
D170G:T198R,MDPGQQPPPQPAPQGQ...,0.877,1,...,-21.221,-6.499,-0.054
D170G:P172L,MDPGQQPPPQPAPQGQ...,0.923,1,...,-19.350,-6.541,-0.052


In [ ]:
# def process_dms(file_path, shuffle=True, max_len=1022, wild_type=None): # There is option to provide the full wild_type aa sequence in str
#     df = pd.read_csv(file_path, index_col='mutant')
#     if shuffle:
#         df = df.sample(frac=1)
   
#     new_df, n_sites = defaultdict(list), set()
#     for mutant, row in df.iterrows(): # row is the entire row... 
#         wt_aas, mt_aas, positions = '', '', []
#         for site in mutant.split(':'): # handle multi-site mutants
#             wt_aa, position, mt_aa = site[0], int(site[1:-1]) - 1, site[-1]
#             if wild_type is None:
#                 assert row['mutated_sequence'][position] == mt_aa # If condition returns False, raise AssertionError # No error handlelation?
#             else:
#                 assert wild_type[position] == wt_aa
#             wt_aas += wt_aa
#             mt_aas += mt_aa
#             positions.append(position)
        
#         new_df['wt_aas'].append(wt_aas)
#         new_df['mt_aas'].append(mt_aas)
#         new_df['positions'].append(tuple(positions))
#         n_sites.add(len(positions)) # Number of mutation sites for each variant 
    
#     new_df = pd.concat([pd.DataFrame(new_df, index=df.index),
#                         df[['DMS_score', 'DMS_score_bin']]], axis=1)
#     if wild_type is None:
#         wild_type = list(row['mutated_sequence'])
#         for wt_aa, position in zip(wt_aas, positions): # recover wild type sequence using the last row
#             wild_type[position] = wt_aa
#         wild_type = ''.join(wild_type)
#     protein = dict(wild_type=wild_type, df=new_df)
#     trunc_sequence(protein, max_len) # Why truncate sequence? 
#     protein['n_sites'] = sorted(n_sites)
#     protein['name'] = os.path.basename(file_path).split('.')[0]
#     return protein

In [7]:
i = 0
wild_type = None

new_df, n_sites = defaultdict(list), set()
for mutant, row in df.iterrows():
    wt_aas, mt_aas, positions = '', '', []
    for site in mutant.split(':'): # handle multi-site mutants
        wt_aa, position, mt_aa = site[0], int(site[1:-1]) - 1, site[-1]
        # print(wt_aa, position, mt_aa)
        wt_aas += wt_aa
        mt_aas += mt_aa
        positions.append(position)

    new_df['wt_aas'].append(wt_aas)
    new_df['mt_aas'].append(mt_aas)
    new_df['positions'].append(tuple(positions))
    n_sites.add(len(positions)) # Number of mutation sites for each variant 

    # i += 1
    # if i == 4:
    #     break
    
new_df = pd.concat([pd.DataFrame(new_df, index=df.index),
                    df[['DMS_score', 'DMS_score_bin']]], axis=1)
    
if wild_type is None:
    wild_type = list(row['mutated_sequence'])
    for wt_aa, position in zip(wt_aas, positions): # recover wild type sequence using the last row
        wild_type[position] = wt_aa
    wild_type = ''.join(wild_type)
protein = dict(wild_type=wild_type, df=new_df)
# trunc_sequence(protein, max_len) # Only carries out if protein wilt_type len is over max_len # How is max_len determined? 
protein['n_sites'] = sorted(n_sites)
protein['name'] = os.path.basename(csv).split('.')[0]
    
new_df

,wt_aas,mt_aas,positions,DMS_score,DMS_score_bin
mutant,,,,,
E178A:M179I,EM,AI,"(177, 178)",0.857,1
E178A:L190R,EL,AR,"(177, 189)",0.659,1
D170K:Q200R,DQ,KR,"(169, 199)",4.720,1
D170G:T198R,DT,GR,"(169, 197)",0.877,1
D170G:P172L,DP,GL,"(169, 171)",0.923,1
...,...,...,...,...,...
Q195K:Q200H,QQ,KH,"(194, 199)",2.976,1
E178N:I193S,EI,NS,"(177, 192)",0.784,1
R187P:Q200E,RQ,PE,"(186, 199)",0.626,1


In [10]:
print(protein.keys())
print(protein['wild_type'])
print(protein['n_sites'])
print(protein['name'])

dict_keys(['wild_type', 'df', 'n_sites', 'name'])
MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQAPPAGHQIVHVRGDSETDLEALFNAVMNPKTANVPQTVPMRLRKLPDSFFKPPEPKSHSRQASTDAGTAGALTPQHVRAHSSPASLQLGAVSPGTLTPTGVVSGPAATPTAQHLRQSSFEIPDDVPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPLPDGWEQAMTQDGEIYYINHKNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSPKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDPFLNSGTYHSRDESTDSGLSMSSYSVPRTPDDFLNSVDEMDTGDTINQSTLPSQQNRFPDYLEAIPGTNVDLGTLEGDGMNIEGEELMPSLQEALSSDILNDMESVLAATKLDKESFLTWL
[1, 2]
YAP1_HUMAN_Araya_2012


In [40]:
trunc_sequence(protein, 1022) 
print(protein.keys())
print(protein['df'].columns)
print(protein['df'])

dict_keys(['wild_type', 'df', 'n_sites', 'name', 'offset'])
Index(['wt_aas', 'mt_aas', 'positions', 'DMS_score', 'DMS_score_bin'], dtype='object')
            wt_aas mt_aas   positions  DMS_score  DMS_score_bin
mutant                                                         
E178A:M179I     EM     AI  (177, 178)      0.857              1
E178A:L190R     EL     AR  (177, 189)      0.659              1
D170K:Q200R     DQ     KR  (169, 199)      4.720              1
D170G:T198R     DT     GR  (169, 197)      0.877              1
D170G:P172L     DP     GL  (169, 171)      0.923              1
...            ...    ...         ...        ...            ...
Q195K:Q200H     QQ     KH  (194, 199)      2.976              1
E178N:I193S     EI     NS  (177, 192)      0.784              1
R187P:Q200E     RQ     PE  (186, 199)      0.626              1
T182Y            T      Y      (181,)      0.867              1
R187S:Q195H     RQ     SH  (186, 194)      0.921              1

[10075 rows x 5 colu

In [18]:
positions = list(chain(*new_df['positions']))
print(positions)
max_pos, min_pos = max(positions), min(positions)
gap = max_pos - min_pos + 1
print(gap)

[177, 178, 177, 189, 169, 199, 169, 197, 169, 171, 177, 200, 169, 177, 181, 192, 185, 192, 171, 194, 186, 194, 180, 199, 171, 181, 177, 195, 180, 186, 169, 177, 188, 192, 182, 193, 170, 194, 188, 185, 189, 171, 186, 173, 182, 199, 183, 199, 173, 199, 170, 200, 190, 197, 170, 174, 179, 188, 177, 188, 179, 190, 177, 190, 170, 174, 193, 174, 201, 179, 201, 177, 180, 175, 200, 170, 195, 178, 179, 169, 198, 193, 199, 177, 183, 179, 200, 171, 177, 180, 184, 197, 183, 193, 174, 188, 189, 171, 185, 171, 197, 178, 200, 183, 200, 169, 180, 173, 181, 194, 195, 194, 199, 193, 200, 171, 180, 185, 192, 182, 192, 171, 187, 174, 185, 170, 185, 177, 189, 191, 199, 170, 199, 177, 192, 186, 192, 196, 177, 198, 197, 199, 175, 200, 175, 192, 174, 192, 186, 193, 180, 194, 172, 199, 188, 192, 172, 202, 171, 174, 180, 175, 201, 182, 185, 178, 184, 187, 202, 192, 178, 182, 171, 178, 180, 189, 193, 194, 188, 202, 177, 192, 178, 196, 190, 192, 174, 183, 170, 178, 193, 195, 193, 197, 170, 171, 173, 199, 193, 200,

In [13]:
# Skipping further breakdown of this function for now. 2024.09.17

# def trunc_sequence(protein, max_len):
#     L = len(protein['wild_type'])
#     if L <= max_len:
#         protein['offset'] = 0
#         return
    
#     df = protein['df']
#     positions = list(chain(*df['positions']))
#     max_pos, min_pos = max(positions), min(positions)
#     gap = max_pos - min_pos + 1
    
#     if max_pos < max_len:
#         protein['wild_type'] = protein['wild_type'][:max_len] # If max_pos is smaller than max_len, remove the sequences exceeding max_len?
#         protein['offset'] = 0
#         return
    
#     if gap <= max_len:
#         window_l = max(min_pos - (max_len - gap) // 2, 0)
#         window_r = min(max_pos + (max_len - gap) // 2, L - 1)
#         seq_lr = protein['wild_type'][window_l: window_l + max_len]
#         seq_rl = protein['wild_type'][window_r - max_len + 1: window_r + 1]
        
#         if len(seq_lr) > len(seq_rl):
#             protein['wild_type'] = seq_lr
#             left, right = window_l, window_l + max_len
#         else:
#             protein['wild_type'] = seq_rl
#             left, right = window_r - max_len + 1, window_r + 1
#     else:
#         n = 0
#         left, right = min_pos, max_len
#         window_l, window_r = min_pos, max_len
#         while window_r < L:
#             window_n = df['positions'].apply(
#                 lambda positions: all(window_l <= pos < window_r for pos in positions)).sum()
#             if window_n > n:
#                 left, right = window_l, window_r
#                 n = window_n
#             window_l += 1
#             window_r += 1
        
#         if right - left + 1 < max_len:
#             left = right - max_len
#         protein['wild_type'] = protein['wild_type'][left:right]
    
#     df_bool = df.apply(lambda row: all(left <= pos < right for pos in row['positions']), axis=1)
#     df = df.loc[df_bool].copy()
#     df.loc[:, 'positions'] = df['positions'].apply(lambda positions: tuple(pos - left for pos in positions))
#     protein['df'] = df
#     protein['offset'] = left
#     return

In [42]:
data_path = os.path.join(folder_path, config.data_path)
print(data_path)
proteins = torch.load(data_path)

sequences = {name: datasets[0]['wild_type'] for name, datasets in proteins.items()}
print(sequences)

/illumina/scratch/SG_PROTEIN_ENG/iPEG/Liuyi/Pro-FSFP/data/merged.pkl
{'SYUA_HUMAN': 'MDVYMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLFVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGYVKKDQLGKNEEGAPQEGILEDMPVDPDNEAFEMPSEEGFQDFEPEA', 'YAP1_HUMAN': 'MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQAPPAGHQIVHVRGDSETDLEALFNAVMNPKTANVPQTVPMRLRKLPDSFFKPPEPKSHSRQASTDAGTAGALTPQHVRAHSSPASLQLGAVSPGTLTPTGVVSGPAATPTAQHLRQSSFEIPDDVPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPLPDGWEQAMTQDGEIYYINHKNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSPKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDPFLNSGTYHSRDESTDSGLSMSSYSVPRTPDDFLNSVDEMDTGDTINQSTLPSQQNRFPDYLEAIPGTNVDLGTLEGDGMNIEGEELMPSLQEALSSDILNDMESVLAATKLDKESFLTWL'}


In [49]:
# Simulate argparse behavior in Jupyter

# Create a class to mimic argparse.Namespace
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

# Define default arguments in a dictionary
args_dict = {
    'mode': 'vectorize',
    'model': 'esm2',
    'pooling': 'average',
    'batch_size': 64,
    'top_k': 10,
    'metric': 'cosine',
    'force_cpu': False,
    'save_postfix': ''
}

# Simulate the parser behavior and return an Args object
def parse_args_notebook():
    # Return an Args object with the simulated arguments
    return Args(**args_dict)

# Now you can access arguments just like you would with argparse
args = parse_args_notebook()

# Example usage
print("Mode:", args.mode)
print("Model:", args.model)
print("Batch Size:", args.batch_size)


Mode: vectorize
Model: esm2
Batch Size: 64


In [50]:
retriever = Retriever(args)
retriever(sequences)

RuntimeError: unable to mmap 2609506392 bytes from file </home/lyi/.cache/huggingface/hub/models--facebook--esm2_t33_650M_UR50D/snapshots/08e4846e537177426273712802403f7ba8261b6c/model.safetensors>: Cannot allocate memory (12)

In [46]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--mode', '-m', type=str, choices=['vectorize', 'retrieve'], default='retrieve',
                        help='compute embedding vectors, or retrieve top-k dms datasets using the saved vectors')
    parser.add_argument('--model', '-md', type=str, choices=config.model_dir.keys(), default='esm1v-1',
                        help='name of the foundation model for vectorizing')
    parser.add_argument('--pooling', '-p', type=str, choices=['average', 'max', 'last'], default='average',
                        help='the method to reduce the model hidden states when vectorizing')
    parser.add_argument('--batch_size', '-b', type=int, default=64,
                        help='batch size for vectorizing and retrieving')
    parser.add_argument('--top_k', '-k', type=int, default=10,
                        help='number of similar datasets to retrieve')
    parser.add_argument('--metric', '-mt', type=str, choices=['cosine', 'l2'], default='cosine',
                        help='similarity metric')
    parser.add_argument('--force_cpu', '-cpu', action='store_true',
                        help='use cpu for vectorizing and retrieving even if gpu is available')
    parser.add_argument('--save_postfix', '-sp', type=str, default='',
                        help='a custom string to append to the data path')
    return parser.parse_args()

args = parse_args()

usage: ipykernel_launcher.py [-h] [--mode {vectorize,retrieve}]
                             [--model {esm1b,esm1v-1,esm1v-2,esm1v-3,esm1v-4,esm1v-5,esm2,saprot}]
                             [--pooling {average,max,last}]
                             [--batch_size BATCH_SIZE] [--top_k TOP_K]
                             [--metric {cosine,l2}] [--force_cpu]
                             [--save_postfix SAVE_POSTFIX]
ipykernel_launcher.py: error: argument --force_cpu/-cpu: ignored explicit argument '/home/lyi/.local/share/jupyter/runtime/kernel-v39fb4683d00544df002713de5697d64fc1a41bca1.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'